In [80]:
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

import numpy as np
import pandas as pd
import csv
import math
from collections import Counter
import matplotlib.pyplot as plt

df=pd.read_csv('C:/Users/geun/KoreanNewsChronicle/Topic_Abstract/Data/201912.csv',encoding='utf-8')

# 결측지 행 제거 후 topics에 있는 것을 추출
s = df.dropna()
tmp_index=[]

for index, row in s.iterrows():
    tmp = row['topics'].split(',')
    if len(tmp) < 2:
        tmp_index.append(index)

s = s.drop(tmp_index)

In [81]:

news_topic = s['topics'].tolist()

# # CountVectrizer로 토큰화
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(news_topic)

X = normalize(X)

neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(X)
distances, indices = nbrs.kneighbors(X)


m = DBSCAN(eps=0.6, min_samples = 5)

m.fit(X)

labels = m.labels_

In [82]:
s['labels'] = labels

hot_topic_index = Counter(s['labels']).most_common()
hot_topic = dict(hot_topic_index)

print(hot_topic)

{-1: 25408, 13: 88, 41: 45, 20: 40, 14: 36, 39: 27, 33: 21, 24: 16, 12: 14, 51: 14, 35: 12, 17: 11, 32: 10, 2: 9, 31: 9, 48: 9, 53: 9, 62: 9, 67: 9, 69: 9, 1: 8, 8: 8, 9: 8, 15: 8, 61: 8, 22: 8, 80: 8, 50: 8, 55: 8, 59: 8, 68: 8, 75: 8, 4: 7, 5: 7, 7: 7, 23: 7, 30: 7, 25: 7, 28: 7, 34: 7, 52: 7, 63: 7, 65: 7, 72: 7, 0: 6, 10: 6, 36: 6, 16: 6, 26: 6, 29: 6, 37: 6, 38: 6, 43: 6, 44: 6, 46: 6, 54: 6, 56: 6, 78: 6, 76: 6, 79: 6, 49: 5, 3: 5, 6: 5, 11: 5, 21: 5, 18: 5, 19: 5, 27: 5, 47: 5, 40: 5, 42: 5, 64: 5, 45: 5, 57: 5, 60: 5, 58: 5, 66: 5, 70: 5, 81: 5, 71: 5, 73: 5, 74: 5, 77: 5}


In [83]:
count = []
for index, row in s['labels'].iteritems():
    if row == -1:
        s = s.drop(index)
    else:
        count.append(hot_topic[row])

s['cnt'] = count
s.sort_values(by='labels', ascending=False).to_csv('./Data/201912_cluster_DBSCAN_10.csv', index=False, header=True,encoding="utf-8-sig")